In [16]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
import sys
sys.path.insert(0, '..')  
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
import torch 
import pandas as pd

In [18]:
from cnn_models import PretrainedAlexNet
from hyperparameter_search import Net_wrapper
import datasets as ds_own

In [19]:
train_dataset = ds_own.cifar_train_fullAN
val_dataset = ds_own.cifar_val_AN
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

In [11]:
my_net = Net_wrapper(model = PretrainedAlexNet, optimizer=optim.Adam, max_epochs=15, learning_rate = 0.0001, step_size=5)
my_net.score(train_dataset, val_dataset, verbose=1)


# Alex net on augmented data

In [21]:
transformer4 = transforms.Compose([transforms.Resize((300, 300)),
                                  transforms.RandomRotation(degrees=(-30,30)),
                                  transforms.ColorJitter(brightness=0.4),
                                  transforms.RandomHorizontalFlip(p=0.5),
                                  transforms.RandomCrop(256),
                                  transforms.ToTensor(),
                                  transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
                                  ds_own.GaussianNoise(mean=0, std=0.001)])

cifar_train_AN_aug = ds_own.CifarDataset(root_dir = ds_own.TRAIN_DIR, labels=ds_own.TRAIN_LABELS, 
                           transform=transformer4, class_dict=ds_own.CLASS_DICT)

my_net = Net_wrapper(model = PretrainedAlexNet, optimizer=optim.Adam, max_epochs=15, learning_rate = 0.0001, step_size=5)

In [ ]:
my_net.score(cifar_train_AN_aug, val_dataset, verbose=1)

In [ ]:
alexNet = PretrainedAlexNet()
alexNet.load_state_dict(torch.load('..\\saved_models\\alex.pt'))
alexNet.eval()

In [3]:
test = ds_own.cifar_test_AN

In [8]:
submission = alexNet.prepare_submission(test, ds_own.CLASS_DICT)
submission.to_csv('submissions\\alexNet_submission.csv', index=False)